In [1]:
import numpy as np
import pandas as pd
import pulp
import itertools
import gmaps
import googlemaps
from math import radians
import scipy as sp
import pymc3 

ModuleNotFoundError: No module named 'pymc3'

In [2]:
from sklearn.neighbors import DistanceMetric

In [1]:
!pip3 list

Package           Version
----------------- -------
arviz             0.11.2
cachetools        4.2.1
cftime            1.4.1
cycler            0.10.0
dill              0.3.3
fastprogress      1.0.0
filelock          3.0.12
kiwisolver        1.3.1
matplotlib        3.4.1
netCDF4           1.5.6
numpy             1.20.2
packaging         20.9
pandas            1.2.4
patsy             0.5.1
Pillow            8.2.0
pip               20.2.3
pymc3             3.11.2
pyparsing         2.4.7
python-dateutil   2.8.1
pytz              2021.1
scipy             1.6.2
semver            2.13.0
setuptools        49.2.1
six               1.15.0
Theano-PyMC       1.1.2
typing-extensions 3.7.4.3
xarray            0.17.0
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [13]:
def manhattan_distmtx(X, Y):
    f = np.dot(X.sum(axis=1).reshape(-1, 1), Y.sum(axis=1).reshape(-1, 1).T)
    return f / Y.sum(axis=1) - Y.sum(axis=1)

In [9]:
# customer count ('0' is depot) 
customer_count = 10

# the number of vehicle
vehicle_count = 4

# the capacity of vehicle
vehicle_capacity = 50

# fix random seed
np.random.seed(seed=777)

# set depot latitude and longitude
depot_latitude = 42.339309
depot_longitude = -71.088735

# make dataframe which contains vending machine location and demand
df = pd.DataFrame({"latitude":np.random.normal(depot_latitude, 0.5, customer_count), 
                   "longitude":np.random.normal(depot_longitude, 0.5, customer_count), 
                   "demand":np.random.randint(10, 20, customer_count)})
df

,latitude,longitude,demand
0,42.105205,-70.816330,13
1,41.927897,-70.141155,14
2,42.306619,-71.473414,10
3,41.982628,-71.790283,16
4,42.792484,-71.404969,15
5,42.722427,-71.368172,17
6,42.752336,-71.705351,15
7,41.677468,-71.308487,15
8,41.463087,-70.631341,18
9,42.840534,-70.956215,11


In [12]:
# set the depot as the center and make demand 0 ('0' = depot)
df.iloc[0,2] = 0
df

,latitude,longitude,demand
0,42.105205,-70.816330,0
1,41.927897,-70.141155,14
2,42.306619,-71.473414,10
3,41.982628,-71.790283,16
4,42.792484,-71.404969,15
5,42.722427,-71.368172,17
6,42.752336,-71.705351,15
7,41.677468,-71.308487,15
8,41.463087,-70.631341,18
9,42.840534,-70.956215,11


In [34]:
# ## function for calculating distance between two pins
# def _distance_calculator(_df):
    
#     _distance_result = np.zeros((len(_df),len(_df)))
#     _df['latitude','longitude'] = '0'
#     for i,j in range(len(_df)):
#         _df['latitude'].iloc[i] = df.latitude[i]
#         _df['longitude'].iloc[j] = df.longitude[i]
    
#     for i in range(len(_df)):
#         for j in range(len(_df)):
            
#             # calculate distance of all pairs
#             _google_maps_api_result = manhattan_distmtx(_df['latitude'].iloc[i],_df['longitude'].iloc[j])
#             # append distance to result list
#             _distance_result[i][j] = _google_maps_api_result[0]['legs'][0]['distance']['value']
    
#     return _distance_result

# distance = _distance_calculator(df)


In [13]:
df['latitude'] = np.radians(df['latitude'])
df['longitude'] = np.radians(df['longitude'])
df
df[['latitude','longitude']].to_numpy()
dist = DistanceMetric.get_metric('haversine')
distance = pd.DataFrame(dist.pairwise(df[['latitude','longitude']].to_numpy())*6373,  columns=df.index.unique(), index=df.index.unique())
distance

,0,1,2,3,4,5,6,7,8,9
0,0.000000,59.178202,58.590450,81.597864,90.431989,82.261157,102.510139,62.643389,73.051992,82.591718
1,59.178202,0.000000,117.714963,136.546198,141.552231,134.131822,157.944769,100.717688,65.805811,121.608596
2,58.590450,117.714963,0.000000,44.515303,54.332978,47.048292,53.097219,71.296217,116.899269,72.948690
3,81.597864,136.546198,44.515303,0.000000,95.480051,89.303661,85.898524,52.408238,112.233254,117.461769
4,90.431989,141.552231,54.332978,95.480051,0.000000,8.351829,24.929125,124.277372,161.050925,37.002206
5,82.261157,134.131822,47.048292,89.303661,8.351829,0.000000,27.746033,116.334691,152.707503,36.105659
6,102.510139,157.944769,53.097219,85.898524,24.929125,27.746033,0.000000,123.946554,168.576162,61.924302
7,62.643389,100.717688,71.296217,52.408238,124.277372,116.334691,123.946554,0.000000,61.186681,132.577825
8,73.051992,65.805811,116.899269,112.233254,161.050925,152.707503,168.576162,61.186681,0.000000,155.537300
9,82.591718,121.608596,72.948690,117.461769,37.002206,36.105659,61.924302,132.577825,155.537300,0.000000


In [15]:
# solve with pulp
for vehicle_count in range(1,vehicle_count+1):
    
    # definition of LpProblem instance
    problem = pulp.LpProblem("CVRP", pulp.LpMinimize)

    # definition of variables which are 0/1
    x = [[[pulp.LpVariable("x%s_%s,%s"%(i,j,k), cat="Binary") if i != j else None for k in range(vehicle_count)]for j in range(customer_count)] for i in range(customer_count)]

    # add objective function
    problem += pulp.lpSum(distance[i][j] * x[i][j][k] if i != j else 0
                          for k in range(vehicle_count) 
                          for j in range(customer_count) 
                          for i in range (customer_count))

    # constraints
    # formula (2)
    for j in range(1, customer_count):
        problem += pulp.lpSum(x[i][j][k] if i != j else 0 
                              for i in range(customer_count) 
                              for k in range(vehicle_count)) == 1 

    # formula (3)
    for k in range(vehicle_count):
        problem += pulp.lpSum(x[0][j][k] for j in range(1,customer_count)) == 1
        problem += pulp.lpSum(x[i][0][k] for i in range(1,customer_count)) == 1

    # formula (4)
    for k in range(vehicle_count):
        for j in range(customer_count):
            problem += pulp.lpSum(x[i][j][k] if i != j else 0 
                                  for i in range(customer_count)) -  pulp.lpSum(x[j][i][k] for i in range(customer_count)) == 0

    #formula (5)
    for k in range(vehicle_count):
        problem += pulp.lpSum(df.demand[j] * x[i][j][k] if i != j else 0 for i in range(customer_count) for j in range (1,customer_count)) <= vehicle_capacity 


    # formula (6)
    subtours = []
    for i in range(2,customer_count):
         subtours += itertools.combinations(range(1,customer_count), i)

    for s in subtours:
        problem += pulp.lpSum(x[i][j][k] if i !=j else 0 for i, j in itertools.permutations(s,2) for k in range(vehicle_count)) <= len(s) - 1

    
    # print vehicle_count which needed for solving problem
    # print calculated minimum distance value
    if problem.solve() == 1:
        print('Vehicle Requirements:', vehicle_count)
        print('Moving Distance:', pulp.value(problem.objective))
        break

Vehicle Requirements: 3
Moving Distance: 738.2050496281752
